In [16]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [17]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_15Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [18]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [19]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [20]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [21]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [22]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    
    global df
    for i in range(15):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-d95d870e84b54a33bc3a569bec89e950","object":"text_completion","created":455946,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 3.476907 seconds 


Prompt==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-0f0a69943a254bc490cc27535b571e06","object":"text_completion","created":455946,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/c

Prompt==> Problem: Write a function in python to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-dac54136e0e044f28bcd4fd261d36ca3","object":"text_completion","created":455946,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ...\n\nThe Fibonacci sequence is defined as follows:\n\nF(0) = 0, F(1) = 1\n\nFor n > 1, F(n) = F(n-1) + F(n-2)\n\nHere is a Python function that generates the nth Fibonacci number:\n\n```python\ndef fibonacci(n):\n    if n <= 0:\n        return \"Input should be a positive integer.\"\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        a, b = 0, 1\n       ","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":43,"total_tokens":243,"completion_tokens":200}}
Execution time: 12.274512 

Prompt==> Problem: Write a function in c++ to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-5cea82cefc864ae0830f834d256681ab","object":"text_completion","created":455953,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n#include <iostream>\n#include <string>\n\nusing namespace std;\n\nstring reverseString(string s) {\n    int len = s.length();\n    string reversed = \"\";\n    for (int i = len - 1; i >= 0; i--) {\n        reversed += s[i];\n    }\n    return reversed;\n}\n\nint main() {\n    string s;\n    cout << \"Enter a string: \";\n    getline(cin, s);\n    cout << \"Reversed string: \" << reverseString(s) << endl;\n    return 0;\n}","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":32,"total_tokens":180,"completion_tokens":148}}
Execution time: 8.808592 seconds 


Prompt==> Problem: Write a function in java to perform bi

Prompt==> Problem: Write a function in c++ to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num. 
 {"id":"cmpl-c309579f2bb54357ba37904871b72a83","object":"text_completion","created":455957,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\n```cpp\n#include <iostream>\nusing namespace std;\n\nint sumOfDigits(int num) {\n    int sum = 0;\n    while (num != 0) {\n        sum += num % 10;\n        num /= 10;\n    }\n    return sum;\n}\n\nint main() {\n    int num;\n    cout << \"Enter a positive integer: \";\n    cin >> num;\n    cout << \"Sum of digits: \" << sumOfDigits(num) << endl;\n    return 0;\n}\n```\n\nThis program first asks the user to input a positive integer. It then calls the function `sumOfDigits` with the input number as the argument. The function calculates the sum of the digits by re

Prompt==> Problem: Write a function in Ruby to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-989d55d6bf2e45dca1ba464db61c5155","object":"text_completion","created":455969,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef factorial(n)\n  if n == 0\n    1\n  else\n    n * factorial(n-1)\n  end\nend\n\nputs factorial(5)","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":41,"total_tokens":88,"completion_tokens":47}}
Execution time: 3.113976 seconds 


Prompt==> Problem: Write a function in Ruby to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-f0fa6d992ddc47f59f1ad8cee619e837","object":"text_completion","created":455970,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c37

...

Prompt==> Problem: Write a function in angular that dynamically creates an HTML element and appends it to the document.Input: String, element type (e.g., 'div', 'button').Output: None (update the HTML document). 
 {"id":"cmpl-c3ef2beada3a4099acb3b5412b090c3f","object":"text_completion","created":456390,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple function in Angular that creates an HTML element and appends it to the document:\n\n```typescript\nimport { Component, ElementRef, Renderer2 } from '@angular/core';\n\n@Component({\n  selector: 'app-root',\n  templateUrl: './app.component.html',\n  styleUrls: ['./app.component.css']\n})\nexport class AppComponent {\n  constructor(private el: ElementRef, private renderer: Renderer2) {}\n\n  createElement(elementType: string, text: string) {\n    const newElement = this.renderer.createElement(elementType);\n    const textNode = t

Prompt==> Problem: Implement form validation in jQuery to ensure that specific fields are filled out correctly.Input: Form input values.Output: Boolean, indicating whether the form is valid. 
 {"id":"cmpl-4781c4285595443f9c5181454aadc967","object":"text_completion","created":456395,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```javascript\nfunction validateForm() {\n    var isValid = true;\n\n    // Validate name field\n    var name = $(\"#name\").val();\n    if (name === \"\") {\n        $(\"#name\").css(\"border\", \"2px solid red\");\n        isValid = false;\n    } else {\n        $(\"#name\").css(\"border\", \"1px solid #ccc\");\n    }\n\n    // Validate email field\n    var email = $(\"#email\").val();\n    var emailReg = /^([\\w-]+(?:\\.[\\w-]+)*)@((?:[\\w-]+\\.)*\\w[\\w-]{0,66})\\.([a-z]{2,6}(?:\\.[a-z]{2})?)$/i;\n    if (email === \"\" || !email.match(emailReg)) {\n        $(\

Prompt==> Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-3dc12905a39e40419e92226a78abfeb9","object":"text_completion","created":456411,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef is_palindrome(s):\n    return s == s[::-1]\n\n# Test the function\nprint(is_palindrome('madam'))  # True\nprint(is_palindrome('hello'))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":39,"total_tokens":98,"completion_tokens":59}}
Execution time: 4.182351 seconds 


Prompt==> Problem: Write a function in python to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-41b2df86c47a4d8a99d5d23a88e2a41c","object":"text_completion","created":456411,"model":"/llmmodels/models--ise-uiuc--M

Prompt==> Problem: Write a function in java to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-2d41d57da2314851a1d56be778ef5632","object":"text_completion","created":456414,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\nHere is a simple Java function to check if two strings are anagrams:\n\n```java\nimport java.util.Arrays;\n\npublic class Main {\n    public static void main(String[] args) {\n        System.out.println(areAnagrams(\"listen\", \"silent\"));  // Output: true\n        System.out.println(areAnagrams(\"hello\", \"world\"));  // Output: false\n    }\n\n    public static boolean areAnagrams(String s1, String s2) {\n        if (s1.length() != s2.len

Prompt==> Problem: Write a function in javascript to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-6f70a049358d49d486b9af2372431a2e","object":"text_completion","created":456427,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction factorial(n) {\n    if (n === 0) {\n        return 1;\n    } else {\n        return n * factorial(n - 1);\n    }\n}\n\nconsole.log(factorial(5)); // Output: 120","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":40,"total_tokens":107,"completion_tokens":67}}
Execution time: 4.447103 seconds 


Prompt==> Problem: Write a function in javascript to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-139d08b78cb44d729c80dfbb9e659376","object":"text_completion","created":456428,"model":"

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))